In [ ]:
import os
import numpy as np

from hmog import HmogHelper
from tqdm import tqdm

dataset_path = 'hmog_public_dataset/'

subjects = HmogHelper.list_folder(dataset_path)

session_maps = {}

for subject in subjects:
    ww_subject_sessions = HmogHelper.filter_user_sessions_by_type(os.path.join(dataset_path, subject), HmogHelper.WW_SESSION_TYPES)
    
    session_maps[subject] = []
    for session in ww_subject_sessions:
        session_maps[subject].append(HmogHelper.read_person_session(os.path.join(dataset_path, subject, session)))
        HmogHelper.preprocess_session_data(session_maps[subject][-1])

In [ ]:
session_hmog_matrices = {}

excluded_subjects = []

for subject in tqdm(session_maps):
    sessions = session_maps[subject]

    assert len(sessions) > 2

    success_1, v1 = HmogHelper.extract_hmog_features(sessions[0], HmogHelper.SENSOR_LIST, HmogHelper.DIMS_LIST)
    success_2, v2 = HmogHelper.extract_hmog_features(sessions[1], HmogHelper.SENSOR_LIST, HmogHelper.DIMS_LIST)

    success_3, test_hmog_vector_1 = HmogHelper.extract_hmog_features(sessions[2], HmogHelper.SENSOR_LIST, HmogHelper.DIMS_LIST)
    success_4, test_hmog_vector_2 = HmogHelper.extract_hmog_features(sessions[3], HmogHelper.SENSOR_LIST, HmogHelper.DIMS_LIST)

    # train_hmog_vector = np.vstack((v1, v2))

    if success_1 and success_2 and success_3 and success_4:
        session_hmog_matrices[subject] = []
        session_hmog_matrices[subject].append(v1)
        session_hmog_matrices[subject].append(v2)
        session_hmog_matrices[subject].append(test_hmog_vector_1)
        session_hmog_matrices[subject].append(test_hmog_vector_2)
    else:
        excluded_subjects.append(subject)

# std_deviations, session_hmog_vector = calc_std_and_scale(session_hmog_vector)

In [ ]:
import pickle

pickled_ds_filename = 'hmog_dataset'

if not os.path.exists(pickled_ds_filename):
    hmog_dataset_file = open(pickled_ds_filename, 'wb')
    pickle.dump(session_hmog_matrices, hmog_dataset_file)
    hmog_dataset_file.close()